## Interactive d3js examples

This notebook contains all the interactive d3js examples in Jupyter for this course. The idea was to put these examples in a separate notebook to make the main notebook a little less convoluted.

The main goal of this notebook is to show that with a Jupyter Ipython notebook and d3js you can make interactive visualizations within an notebook environment. This makes it great for explorative research and experiments on all kinds of different python frameworks or mathematical models.

This notebook contains the following to examples (click to jump to the subject):
- [Interactive parallel coordinate plot.](#ipcp)
- [Interactive graph visualisation.](#igv)

## Interactive parallel coordinate plot

With matplotlib and pandas there is no way to select or highlight values in the parallel coordinate plot which can be an insightful interactive tool. Here is an example of this kind if interaction in javascript with the [D3js library](http://d3js.org/).

#### Iris dataset
First we need to create and save Fisher's Iris dataframe to the server.

In [ ]:
# First import all the needed packages and modules
import matplotlib                      # Matplotlib is the core package to make it easy to plot publication grade images
import matplotlib.pyplot as plt        # Pyplot is a collection of command style functions that make matplotlib work like MATLAB
import pandas as pd                    # Pandas is the data analysis and 'visualization' package 
from pandas.tools.plotting import parallel_coordinates # Import the plotting function for the parallel coordinate plot
from sklearn import datasets

# First lets load the Fisher Iris dataset
irisDataset    = datasets.load_iris()

# Now lets prepare the Fisher Iris dataset to the dataframe format of pandas
irisDataFrame  = pd.DataFrame(data=irisDataset.data, columns=irisDataset.feature_names);
speciesNames   = irisDataset.target_names;
irisDataFrame['speciesByName'] = [speciesNames[x] for x in irisDataset.target];

# Now we save the dataframe
irisDataFrame.to_csv('./data/htmlContent/iris.csv');

Now that the data is prepared and stored we can make the html code and write it to a file with %%writefile (for reference the %% sign is called a [cell magic command](https://ipython.org/ipython-doc/3/interactive/magics.html#cell-magics)) so that we can load it with the Jupyter display and IFrame function.

In [ ]:
%%writefile ./data/htmlContent/parallelCoordPlot.html
<html lang="en">
    <head>
        <meta charset="utf-8">
        <style type="text/css">

        svg {
          font: 10px sans-serif;
        }

        canvas, svg {
          position: absolute;
          top: 0;
          left: 0;
        }

        #chart {
          position: relative;
        }

        .brush .extent {
          fill-opacity: .3;
          stroke: #fff;
          shape-rendering: crispEdges;
        }

        .axis line, .axis path {
          fill: none;
          stroke: #000;
          shape-rendering: crispEdges;
        }

        .axis text {
          text-shadow: 0 1px 0 #fff;
        }

        .legend {
              font-size: 18px;
              font-style: oblique;
            }

            .legend line {
              stroke-width: 2px;
            }

            
        </style>
    </head>
    <body>
    <div id="chart">
      <canvas id="background"></canvas>
      <canvas id="foreground"></canvas>
      <svg></svg>
    </div>
        <script src="https://d3js.org/d3.v3.min.js"></script>
        <script>
            var species = ["setosa", "versicolor", "virginica"],
                traits = ["sepal length (cm)", "petal length (cm)", "sepal width (cm)", "petal width (cm)"];

            var m = [30, 10, 10, 10],
                    w = 960 - m[1] - m[3],
                    h = 500 - m[0] - m[2];

            var x = d3.scale.ordinal().rangePoints([0, w], 1),
                y = {};

            var line = d3.svg.line(),
                axis = d3.svg.axis().orient("left"),
                background,
                foreground;

            d3.selectAll("canvas")
                .attr("width", w + m[1] + m[3])
                .attr("height", h + m[0] + m[2])
                .style("padding", m.join("px ") + "px");

            foreground = document.getElementById('foreground').getContext('2d');
            background = document.getElementById('background').getContext('2d');

            foreground.strokeStyle = "rgba(0,100,160,0.24)";
            background.strokeStyle = "rgba(0,0,0,0.02)";    

            var svg = d3.select("svg")
              .attr("width", w + m[1] + m[3])
              .attr("height", h + m[0] + m[2])
              .append("svg:g")
                .attr("transform", "translate(" + m[3] + "," + m[0] + ")");

            d3.csv("./iris.csv", function(flowers) { //Already in htmlContent because we are running from Iframe
              // Create a scale and brush for each trait.

              x.domain(traits);



              traits.forEach(function(d) {
                // Coerce values to numbers.
                flowers.forEach(function(p) { p[d] = parseFloat(p[d]); });

                y[d] = d3.scale.linear()
                    .domain(d3.extent(flowers, function(p) { return p[d]; }))
                    .range([h, 0]);
             });

              flowers.map(function(d) {
                path(d, "background");
                path(d, "foreground");
              });

              //Add a legend.
              var legend = svg.selectAll("g.legend")
                  .data(species)
                .enter().append("svg:g")
                  .attr("class", "legend")
                  .attr("transform", function(d, i) { return "translate(0," + (i * 20 + 584) + ")"; });

              legend.append("svg:line")
                  .attr("class", String)
                  .attr("x2", 8);

              legend.append("svg:text")
                  .attr("x", 12)
                  .attr("dy", ".31em")
                  .text(function(d) { return "Iris " + d; });

              // Add a group element for each trait.
              var g = svg.selectAll(".trait")
                  .data(traits)
                .enter().append("svg:g")
                  .attr("class", "trait")
                  .attr("transform", function(d) { return "translate(" + x(d) + ")"; })

              // Add an axis and title.
              g.append("svg:g")
                  .attr("class", "axis")
                  .each(function(d) { d3.select(this).call(axis.scale(y[d])); })
                .append("svg:text")
                  .attr("text-anchor", "middle")
                  .attr("y", -9)
                  .text(String);

              // Add a brush for each axis.
              g.append("svg:g")
                  .attr("class", "brush")
                  .each(function(d) { d3.select(this).call(y[d].brush = d3.svg.brush().y(y[d]).on("brush", brush)); })
                .selectAll("rect")
                  .attr("x", -8)
                  .attr("width", 16);


              // Handles a brush event, toggling the display of foreground lines.
              function brush() {
                var actives = traits.filter(function(p) { return !y[p].brush.empty(); }),
                    extents = actives.map(function(p) { return y[p].brush.extent(); });

                // Get lines within extents
                var selected = [];
                flowers.map(function(d) {
                  return actives.every(function(p, i) {
                    return extents[i][0] <= d[p] && d[p] <= extents[i][1];
                  }) ? selected.push(d) : null;
                });

                // Render selected lines
                foreground.clearRect(0,0,w+1,h+1);
                selected.map(function(d) {
                  path(d, 'foreground');
                });
              }
            });
            
            // This function draws the lines of the parallel coordinate plot
            function path(d, ctx) {
              c = ctx === 'foreground' ? foreground : background; 
              if(ctx === 'foreground'){
                switch(d.speciesByName){
                  case 'setosa': c.strokeStyle = "rgba(136,0,0,0.24)";
                  break;
                  case 'versicolor': c.strokeStyle = "rgba(0,136,0,0.24)";
                  break;
                  case 'virginica': c.strokeStyle = "rgba(0,0,136,0.24)";
                }  
              }
               
              c.beginPath();
              traits.map(function(p,i) {
                if (i == 0) {
                  c.moveTo(x(p),y[p](d[p]));
                } else { 
                  c.lineTo(x(p),y[p](d[p]));
                }
              });
              c.stroke();
            };
        </script>
    </body>
</html>

As you can see that is quite a lot more code than the python equivalent but by executing the following cell the difference interaction can make will reveal itself.

In [ ]:
from IPython.display import display, IFrame

display(IFrame('./data/htmlContent/parallelCoordPlot.html',900,700))

If all went well you should be seeing a parallel coordinate plot like the one created with in python with Pandas and Matplotlib. There are some differences, i.e. the axes are scaled according the minimum and maximum values of the features and the you can interact with the axes. This interaction is the main benefit where you can 'brush' the axes and highlight the lines that are running through that brush. For this dataset for example you can highlight a range on sepal width axis and the petal width axes and see how the entries passing through these ranges are represented on the other axes without any visual clutter of the other lines.

#### HR resources
We can do the same thing for the HR resources dataset.

In [ ]:
# First lets clear all previous python imports and variables by resetting the python kernel with the line magic command %reset.

%matplotlib nbagg

# First import all the needed packages and modules
import matplotlib                      # Matplotlib is the core package to make it easy to plot publication grade images
import matplotlib.pyplot as plt        # Pyplot is a collection of command style functions that make matplotlib work like MATLAB
import pandas as pd                    # Pandas is the data analysis and 'visualization' package 
from pandas.tools.plotting import parallel_coordinates # Import the plotting function for the parallel coordinate plot
from sklearn import datasets
from IPython.display import display, HTML, Javascript

# First lets load the Fisher Iris dataset
HRresources = pd.read_csv('./data/parallelCoord/HR_comma_sep_6000.csv')
HRresources.drop(['sales'], axis=1).to_csv('./data/htmlContent/hrresources.csv');

In [ ]:
%%writefile ./data/htmlContent/parallelCoordPlotHRRecources.html
<html lang="en">
    <head>
        <meta charset="utf-8">
        <style type="text/css">
                               svg {
          font: 10px sans-serif;
        }

        canvas, svg {
          position: absolute;
          top: 0;
          left: 0;
        }

        #chart {
          position: relative;
        }

        .brush .extent {
          fill-opacity: .3;
          stroke: #fff;
          shape-rendering: crispEdges;
        }

        .axis line, .axis path {
          fill: none;
          stroke: #000;
          shape-rendering: crispEdges;
        }

        .axis text {
          text-shadow: 0 1px 0 #fff;
        }

        .legend {
              font-size: 18px;
              font-style: oblique;
            }

            .legend line {
              stroke-width: 2px;
            }

            
        </style>
    </head>
    <body>
        <div id="chart">
      <canvas id="background"></canvas>
      <canvas id="foreground"></canvas>
      <svg></svg>
    </div>
        <script src="https://d3js.org/d3.v3.min.js"></script>
        <script>
            var species = ["low", "medium", "high"],
                traits = ["satisfaction_level", "last_evaluation", "number_project", "average_montly_hours", 
                          "time_spend_company", "Work_accident", "left", "promotion_last_5years"];
            var m = [30, 10, 10, 10],
                    w = 960 - m[1] - m[3],
                    h = 500 - m[0] - m[2];

            var x = d3.scale.ordinal().rangePoints([0, w], 1),
                y = {};

            var line = d3.svg.line(),
                axis = d3.svg.axis().orient("left"),
                background,
                foreground;

            d3.selectAll("canvas")
                .attr("width", w + m[1] + m[3])
                .attr("height", h + m[0] + m[2])
                .style("padding", m.join("px ") + "px");

            foreground = document.getElementById('foreground').getContext('2d');
            background = document.getElementById('background').getContext('2d');

            foreground.strokeStyle = "rgba(0,100,160,0.24)";
            background.strokeStyle = "rgba(0,0,0,0)";    

            var svg = d3.select("svg")
              .attr("width", w + m[1] + m[3])
              .attr("height", h + m[0] + m[2])
              .append("svg:g")
                .attr("transform", "translate(" + m[3] + "," + m[0] + ")");

            d3.csv("./hrresources.csv", function(flowers) { //Already in htmlContent because we are running from Iframe
              // Create a scale and brush for each trait.

              x.domain(traits);



              traits.forEach(function(d) {
                // Coerce values to numbers.
                flowers.forEach(function(p) { p[d] = parseFloat(p[d]); });

                y[d] = d3.scale.linear()
                    .domain(d3.extent(flowers, function(p) { return p[d]; }))
                    .range([h, 0]);
             });

              flowers.map(function(d) {
                path(d, "background");
                path(d, "foreground");
              });

              //Add a legend.
              var legend = svg.selectAll("g.legend")
                  .data(species)
                .enter().append("svg:g")
                  .attr("class", "legend")
                  .attr("transform", function(d, i) { return "translate(0," + (i * 20 + 584) + ")"; });

              legend.append("svg:line")
                  .attr("class", String)
                  .attr("x2", 8);

              legend.append("svg:text")
                  .attr("x", 12)
                  .attr("dy", ".31em")
                  .text(function(d) { return "Iris " + d; });

              // Add a group element for each trait.
              var g = svg.selectAll(".trait")
                  .data(traits)
                .enter().append("svg:g")
                  .attr("class", "trait")
                  .attr("transform", function(d) { return "translate(" + x(d) + ")"; })

              // Add an axis and title.
              g.append("svg:g")
                  .attr("class", "axis")
                  .each(function(d) { d3.select(this).call(axis.scale(y[d])); })
                .append("svg:text")
                  .attr("text-anchor", "middle")
                  .attr("y", -9)
                  .text(String);

              // Add a brush for each axis.
              g.append("svg:g")
                  .attr("class", "brush")
                  .each(function(d) { d3.select(this).call(y[d].brush = d3.svg.brush().y(y[d]).on("brush", brush)); })
                .selectAll("rect")
                  .attr("x", -8)
                  .attr("width", 16);


              // Handles a brush event, toggling the display of foreground lines.
              function brush() {
                var actives = traits.filter(function(p) { return !y[p].brush.empty(); }),
                    extents = actives.map(function(p) { return y[p].brush.extent(); });

                // Get lines within extents
                var selected = [];
                flowers.map(function(d) {
                  return actives.every(function(p, i) {
                    return extents[i][0] <= d[p] && d[p] <= extents[i][1];
                  }) ? selected.push(d) : null;
                });

                // Render selected lines
                foreground.clearRect(0,0,w+1,h+1);
                selected.map(function(d) {
                  path(d, 'foreground');
                });
              }
            });
    
            // This function draws the lines of the parallel coordinate plot
            function path(d, ctx) {
              c = ctx === 'foreground' ? foreground : background; 
              if(ctx === 'foreground'){
                switch(d.salary){
                  case 'low': c.strokeStyle = "rgba(136,0,0,0.24)";
                  break;
                  case 'medium': c.strokeStyle = "rgba(0,136,0,0.24)";
                  break;
                  case 'high': c.strokeStyle = "rgba(0,0,136,0.24)";
                }  
              }
               
              c.beginPath();
              traits.map(function(p,i) {
                if (i == 0) {
                  c.moveTo(x(p),y[p](d[p]));
                } else { 
                  c.lineTo(x(p),y[p](d[p]));
                }
              });
              c.stroke();
            };
           
        </script>
    </body>
</html>

In [ ]:
from IPython.display import display, IFrame

display(IFrame('./data/htmlContent/parallelCoordPlotHRRecources.html',900,700))

#### Oldtimer car dataset
We can do the same thing for the oldtimer car dataset.

In [ ]:
import matplotlib               # Matplotlib is the core package to plot publication grade images
import matplotlib.pyplot as plt # # Pyplot: a collection functions that make matplotlib work like MATLAB
import pandas as pd             # Pandas is the data analysis and 'visualization' package\
import numpy as np
from pandas.tools.plotting import scatter_matrix # Import the function to plot a scatterplot
    
%matplotlib nbagg

cars = pd.read_csv('./data/scatter/cars.csv')
cars.drop(['model'], axis=1).to_csv('./data/htmlContent/cars.csv');

In [ ]:
%%writefile ./data/htmlContent/parallelCoordPlotcars.html
<html lang="en">
    <head>
        <meta charset="utf-8">
        <style type="text/css">
                               svg {
          font: 10px sans-serif;
        }

        canvas, svg {
          position: absolute;
          top: 0;
          left: 0;
        }

        #chart {
          position: relative;
        }

        .brush .extent {
          fill-opacity: .3;
          stroke: #fff;
          shape-rendering: crispEdges;
        }

        .axis line, .axis path {
          fill: none;
          stroke: #000;
          shape-rendering: crispEdges;
        }

        .axis text {
          text-shadow: 0 1px 0 #fff;
        }

        .legend {
              font-size: 18px;
              font-style: oblique;
            }

            .legend line {
              stroke-width: 2px;
            }

            
        </style>
    </head>
    <body>
        <div id="chart">
      <canvas id="background"></canvas>
      <canvas id="foreground"></canvas>
      <svg></svg>
    </div>
        <script src="https://d3js.org/d3.v3.min.js"></script>
        <script>
            var species = ["US", "Europe", "Japan"],
                traits = ["MPG", "cylinders", "horsepower", "weigth", "year"];
                
            var m = [30, 10, 10, 10],
                    w = 960 - m[1] - m[3],
                    h = 500 - m[0] - m[2];

            var x = d3.scale.ordinal().rangePoints([0, w], 1),
                y = {};

            var line = d3.svg.line(),
                axis = d3.svg.axis().orient("left"),
                background,
                foreground;

            d3.selectAll("canvas")
                .attr("width", w + m[1] + m[3])
                .attr("height", h + m[0] + m[2])
                .style("padding", m.join("px ") + "px");

            foreground = document.getElementById('foreground').getContext('2d');
            background = document.getElementById('background').getContext('2d');

            foreground.strokeStyle = "rgba(0,100,160,0.24)";
            background.strokeStyle = "rgba(0,0,0,0.02)";    

            var svg = d3.select("svg")
              .attr("width", w + m[1] + m[3])
              .attr("height", h + m[0] + m[2])
              .append("svg:g")
                .attr("transform", "translate(" + m[3] + "," + m[0] + ")");

            d3.csv("./cars.csv", function(flowers) { //Already in htmlContent because we are running from Iframe
              // Create a scale and brush for each trait.

              x.domain(traits);



              traits.forEach(function(d) {
                // Coerce values to numbers.
                flowers.forEach(function(p) { p[d] = parseFloat(p[d]); });

                y[d] = d3.scale.linear()
                    .domain(d3.extent(flowers, function(p) { return p[d]; }))
                    .range([h, 0]);
             });

              flowers.map(function(d) {
                path(d, "background");
                path(d, "foreground");
              });

              //Add a legend.
              var legend = svg.selectAll("g.legend")
                  .data(species)
                .enter().append("svg:g")
                  .attr("class", "legend")
                  .attr("transform", function(d, i) { return "translate(0," + (i * 20 + 584) + ")"; });

              legend.append("svg:line")
                  .attr("class", String)
                  .attr("x2", 8);

              legend.append("svg:text")
                  .attr("x", 12)
                  .attr("dy", ".31em")
                  .text(function(d) { return "Iris " + d; });

              // Add a group element for each trait.
              var g = svg.selectAll(".trait")
                  .data(traits)
                .enter().append("svg:g")
                  .attr("class", "trait")
                  .attr("transform", function(d) { return "translate(" + x(d) + ")"; })

              // Add an axis and title.
              g.append("svg:g")
                  .attr("class", "axis")
                  .each(function(d) { d3.select(this).call(axis.scale(y[d])); })
                .append("svg:text")
                  .attr("text-anchor", "middle")
                  .attr("y", -9)
                  .text(String);

              // Add a brush for each axis.
              g.append("svg:g")
                  .attr("class", "brush")
                  .each(function(d) { d3.select(this).call(y[d].brush = d3.svg.brush().y(y[d]).on("brush", brush)); })
                .selectAll("rect")
                  .attr("x", -8)
                  .attr("width", 16);


              // Handles a brush event, toggling the display of foreground lines.
              function brush() {
                var actives = traits.filter(function(p) { return !y[p].brush.empty(); }),
                    extents = actives.map(function(p) { return y[p].brush.extent(); });

                // Get lines within extents
                var selected = [];
                flowers.map(function(d) {
                  return actives.every(function(p, i) {
                    return extents[i][0] <= d[p] && d[p] <= extents[i][1];
                  }) ? selected.push(d) : null;
                });

                // Render selected lines
                foreground.clearRect(0,0,w+1,h+1);
                selected.map(function(d) {
                  path(d, 'foreground');
                });
              }
            });

            // This function draws the lines of the parallel coordinate plot
            function path(d, ctx) {
              c = ctx === 'foreground' ? foreground : background; 
              if(ctx === 'foreground'){
                switch(d.origin){
                  case 'US': c.strokeStyle = "rgba(136,0,0,0.24)";
                  break;
                  case 'Europe': c.strokeStyle = "rgba(0,136,0,0.24)";
                  break;
                  case 'Japan': c.strokeStyle = "rgba(0,0,136,0.24)";
                }  
              }
               
              c.beginPath();
              traits.map(function(p,i) {
                if (i == 0) {
                  c.moveTo(x(p),y[p](d[p]));
                } else { 
                  c.lineTo(x(p),y[p](d[p]));
                }
              });
              c.stroke();
            };
           
        </script>
    </body>
</html>

In [ ]:
from IPython.display import display, IFrame

display(IFrame('./data/htmlContent/parallelCoordPlotcars.html',900,700))

<a id='igv'></a>
## Interactive graph visualization

In the main course notebook we have seen some of the capabilities of the python package Networkx we will now see an example of the same graph but then implemented with D3js to add some interactivity.

First we need to create the graph.

In [ ]:
# First lets clear all previous python imports and variables by resetting the python kernel.
#%reset -f

%matplotlib nbagg

import matplotlib                      # Matplotlib is the core package to make it easy to plot publication grade images
import matplotlib.pyplot as plt        # Pyplot is a collection of command style functions that make matplotlib work like MATLAB
import networkx as nx                  # The network graph visualisation module

g = nx.read_graphml('./data/networks/networkcitnetwork.graphml') # Read the graphml file from the server and format to a networkx graph

# A list for the size of every node in the graph based on their degree (the amount of connected neighbours)
nodeSizes  = [10*(len(g[node[0]])+1) for node in g.nodes(data=True)] # We add one or else nodes with no neighbours would have size 0
# A list for the color of every node in the graph based on their degree (the amount of connected neighbours)
nodeValuesColorMapping = [10*len(g[node[0]]) for node in g.nodes(data=True)] 
# A dictionary with key:nodeId and value:labelname to label each node in the visualisation
labels     = {node[0]: node[1]['label'] for node in g.nodes(data=True)} 

 And then save it to the server and import the d3js library in this jupyter enviroment.

In [ ]:
from networkx.readwrite import json_graph
import json

g.graph['MinMaxNodeColorValues'] = [min(nodeValuesColorMapping),max(nodeValuesColorMapping)];

for i, node in enumerate(g.nodes(data=True)):
    node[1]['size'] = nodeSizes[i];
    
# Write to json format for d3js to read
jsonFile = open('./data/htmlContent/networkcitnetwork.json', 'w');
jsonFile.write(json.dumps(json_graph.node_link_data(g)));

Now lets do some HTML magic to implement the CSS style sheet.

In [ ]:
%%writefile ./data/htmlContent/graph.html
<html lang="en">
    <head>
        <meta charset="utf-8">
        <style type="text/css">
            .node {
              stroke: #fff;
              stroke-width: 1.5px;
            }

            .link {
              stroke: #999;
              stroke-opacity: .6;
            }
        </style>
    </head>
    <body>
        <script src="https://d3js.org/d3.v3.min.js"></script>
        <script>
            var width = 1000,
                height = 1000;

            var color = d3.scale.linear()
                .domain([0, 1])
                .range(["#fff5f0", "#67000d"]);

            var force = d3.layout.force()
                .charge(-15)
                .linkDistance(2.5)
                .size([width, height]);

            var svg = d3.select("body").append("svg")
                .attr("width", width)
                .attr("height", height);

            d3.json("./networkcitnetwork.json", function(error, graph) {
              if (error) throw error;

              color.domain(graph.graph.MinMaxNodeColorValues);

              force
                  .nodes(graph.nodes)
                  .links(graph.links)
                  .start();

              var link = svg.selectAll(".link")
                  .data(graph.links)
                .enter().append("line")
                  .attr("class", "link");

              var node = svg.selectAll(".node")
                  .data(graph.nodes)
                .enter().append("circle")
                  .attr("class", "node")
                  .attr("r", function(d){ return Math.sqrt(d.size/Math.PI); })
                  .style("fill", function(d) { return color(d.size); })
                  .call(force.drag);

              node.append("title")
                  .text(function(d) { return d.label; });

              force.on("tick", function() {
                link.attr("x1", function(d) { return d.source.x; })
                    .attr("y1", function(d) { return d.source.y; })
                    .attr("x2", function(d) { return d.target.x; })
                    .attr("y2", function(d) { return d.target.y; });

                node.attr("cx", function(d) { return d.x; })
                    .attr("cy", function(d) { return d.y; });
              });
            });
        </script>
    </body>
</html>
        

Finally some javascript magic to implement the javascript visualization.

In [ ]:
from IPython.display import display, IFrame

display(IFrame('./data/htmlContent/graph.html',950,800))

Ofcourse this is just a simple example and there are many more advanced visualizations and ways of interaction within D3js itself. For more examples you can take a look at the [D3js example page](https://github.com/mbostock/d3/wiki/Gallery).